In [70]:
import pandas as pd
import requests
import json
from datetime import datetime
from urllib.parse import urlencode
from dotenv import load_dotenv
from requests import get
from json import dumps
import sqlalchemy as db

In [103]:
response = requests.get('https://api.coronavirus.data.gov.uk/v1/data?filters=areaType=region&structure={"date":"date","name":"areaName","code":"areaCode","dailyCases":"newCasesByPublishDate","cumulativeCases":"cumCasesByPublishDate","dailyDeaths":"newDeaths28DaysByPublishDate","cumulativeDeaths":"cumDeaths28DaysByPublishDate"}')

In [116]:
response_wales = requests.get('https://api.coronavirus.data.gov.uk/v1/data?filters=areaType=nation;areaName=Wales&structure={"date":"date","name":"areaName","code":"areaCode","dailyCases":"newCasesByPublishDate","cumulativeCases":"cumCasesByPublishDate","dailyDeaths":"newDeaths28DaysByPublishDate","cumulativeDeaths":"cumDeaths28DaysByPublishDate"}')

In [117]:
response_wales

<Response [200]>

In [104]:
response

<Response [200]>

In [68]:
income = pd.read_csv('totalincome.csv')

In [81]:
income['Region name'].unique()

array(['North East', 'North West', 'Yorkshire', 'East Midla',
       'West Midla', 'East', 'London', 'South East', 'South West',
       'Wales'], dtype=object)

In [105]:
response = response.json()
response

{'length': 4572,
 'maxPageLimit': 2500,
 'totalRecords': 11522,
 'data': [{'date': '2021-09-06',
   'name': 'North East',
   'code': 'E12000001',
   'dailyCases': 1628,
   'cumulativeCases': 342144,
   'dailyDeaths': 0,
   'cumulativeDeaths': 6357},
  {'date': '2021-09-05',
   'name': 'North East',
   'code': 'E12000001',
   'dailyCases': 1263,
   'cumulativeCases': None,
   'dailyDeaths': 1,
   'cumulativeDeaths': 6357},
  {'date': '2021-09-04',
   'name': 'North East',
   'code': 'E12000001',
   'dailyCases': 1598,
   'cumulativeCases': None,
   'dailyDeaths': 8,
   'cumulativeDeaths': 6356},
  {'date': '2021-09-03',
   'name': 'North East',
   'code': 'E12000001',
   'dailyCases': 1332,
   'cumulativeCases': None,
   'dailyDeaths': 4,
   'cumulativeDeaths': 6348},
  {'date': '2021-09-02',
   'name': 'North East',
   'code': 'E12000001',
   'dailyCases': 1726,
   'cumulativeCases': None,
   'dailyDeaths': 10,
   'cumulativeDeaths': 6344},
  {'date': '2021-09-01',
   'name': 'North Ea

In [138]:
data_england = pd.DataFrame(response['data'])
data_england['name'].unique()

array(['North East', 'North West', 'Yorkshire and The Humber',
       'East Midlands', 'West Midlands', 'East of England', 'London',
       'South East', 'South West'], dtype=object)

In [120]:
results_wales = response_wales.json()
results_wales


{'length': 557,
 'maxPageLimit': 2500,
 'totalRecords': 2188,
 'data': [{'date': '2021-09-06',
   'name': 'Wales',
   'code': 'W92000004',
   'dailyCases': 5161,
   'cumulativeCases': 293806,
   'dailyDeaths': 8,
   'cumulativeDeaths': 5699},
  {'date': '2021-09-05',
   'name': 'Wales',
   'code': 'W92000004',
   'dailyCases': 2389,
   'cumulativeCases': 288645,
   'dailyDeaths': 3,
   'cumulativeDeaths': 5691},
  {'date': '2021-09-04',
   'name': 'Wales',
   'code': 'W92000004',
   'dailyCases': 0,
   'cumulativeCases': 286256,
   'dailyDeaths': 0,
   'cumulativeDeaths': 5688},
  {'date': '2021-09-03',
   'name': 'Wales',
   'code': 'W92000004',
   'dailyCases': 2391,
   'cumulativeCases': 286256,
   'dailyDeaths': 6,
   'cumulativeDeaths': 5688},
  {'date': '2021-09-02',
   'name': 'Wales',
   'code': 'W92000004',
   'dailyCases': 2275,
   'cumulativeCases': 283865,
   'dailyDeaths': 5,
   'cumulativeDeaths': 5682},
  {'date': '2021-09-01',
   'name': 'Wales',
   'code': 'W92000004',

In [140]:
data_wales = pd.DataFrame(results_wales['data'])
data_wales['name'].unique()

array(['Wales'], dtype=object)

In [127]:
data_uk = pd.concat([data_england, data_wales])
data_uk

,date,name,code,dailyCases,cumulativeCases,dailyDeaths,cumulativeDeaths
0,2021-09-06,North East,E12000001,1628,342144.0,0.0,6357.0
1,2021-09-05,North East,E12000001,1263,NaN,1.0,6357.0
2,2021-09-04,North East,E12000001,1598,NaN,8.0,6356.0
3,2021-09-03,North East,E12000001,1332,NaN,4.0,6348.0
4,2021-09-02,North East,E12000001,1726,NaN,10.0,6344.0
...,...,...,...,...,...,...,...
552,2020-03-03,Wales,W92000004,0,1.0,NaN,NaN
553,2020-03-02,Wales,W92000004,0,1.0,NaN,NaN
554,2020-03-01,Wales,W92000004,0,1.0,NaN,NaN
555,2020-02-29,Wales,W92000004,0,1.0,NaN,NaN


In [2]:
import sqlalchemy as db

In [3]:
db_server = 'postgresql'
user = 'postgres'
password = '1234'
ip = 'localhost'
db_name = 'Covid-Project'


engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}:5433/{db_name}')
conn = engine.connect()
#conn.close()

In [137]:
engine.table_names()

[]